# Brain surface visualization

This tutorial will show you how to visualize neuroimaging data on brain surfaces.

@author: Makliya Mamat (makliyamamat@gmail\.com)


- Understanding brain surface data and annotation files
- Working with parcellation schemes and region names
- Mapping data to brain vertices
- Creating single and multi-view brain visualizations
- Customizing colors, views, and layouts




## Setting up and understanding the basics

First, let's import the necessary libraries and understand what we're working with.


In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from brainspace.plotting import plot_surf
from brainstat.datasets import fetch_template_surface
import nibabel as nib
from PIL import Image

# Set up matplotlib for better plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12


## Understanding brain surfaces

Brain surfaces are 3D meshes that represent the cortical surface. The most commonly used template is the fsaverage surface from FreeSurfer. Let's load it and understand its structure.


In [24]:
# Load the fsaverage brain surface template
# This gives us two surfaces: left and right hemispheres
pial_left, pial_right = fetch_template_surface("fsaverage", join=False)

print("Left hemisphere surface:")
print(f"  Number of vertices: {pial_left.n_points}")
print(f"  Number of faces: {pial_left.n_cells}")
print(f"  Coordinates shape: {pial_left.Points.shape}")

print("\nRight hemisphere surface:")
print(f"  Number of vertices: {pial_right.n_points}")
print(f"  Number of faces: {pial_right.n_cells}")
print(f"  Coordinates shape: {pial_right.Points.shape}")


Left hemisphere surface:
  Number of vertices: 163842
  Number of faces: 327680
  Coordinates shape: (163842, 3)

Right hemisphere surface:
  Number of vertices: 163842
  Number of faces: 327680
  Coordinates shape: (163842, 3)


**Key Concepts:**
- **Vertices**: Points in 3D space that define the surface
- **Faces**: Triangular polygons connecting vertices to form the mesh
- **Coordinates**: 3D positions (x, y, z) of each vertex

Each vertex can have associated data (like activation values, connectivity measures, etc.)


## Understanding annotation files

Annotation files (.annot) contain information about which brain regions each vertex belongs to. Let's explore this in detail.


In [25]:
# Load parcellation labels from annotation files
# These files tell us which region each vertex belongs to
labels_lh, _, names_lh = nib.freesurfer.read_annot('parcs/lh.500.aparc.annot')
labels_rh, _, names_rh = nib.freesurfer.read_annot('parcs/rh.500.aparc.annot')

print("Left hemisphere annotation file:")
print(f"  Number of vertices with labels: {len(labels_lh)}")
print(f"  Number of unique regions: {len(names_lh)}")
print(f"  Label range: {labels_lh.min()} to {labels_lh.max()}")

print("\nRight hemisphere annotation file:")
print(f"  Number of vertices with labels: {len(labels_rh)}")
print(f"  Number of unique regions: {len(names_rh)}")
print(f"  Label range: {labels_rh.min()} to {labels_rh.max()}")


Left hemisphere annotation file:
  Number of vertices with labels: 163842
  Number of unique regions: 154
  Label range: 0 to 153

Right hemisphere annotation file:
  Number of vertices with labels: 163842
  Number of unique regions: 158
  Label range: 0 to 157


In [26]:
# Let's look at the region names (they're stored as byte strings)
print("First 10 left hemisphere region names:")
for i, name in enumerate(names_lh[:10]):
    print(f"  {i}: {name}")

print("\nFirst 10 right hemisphere region names:")
for i, name in enumerate(names_rh[:10]):
    print(f"  {i}: {name}")


First 10 left hemisphere region names:
  0: b'unknown_part1'
  1: b'bankssts_part1'
  2: b'bankssts_part2'
  3: b'caudalanteriorcingulate_part1'
  4: b'caudalmiddlefrontal_part1'
  5: b'caudalmiddlefrontal_part2'
  6: b'caudalmiddlefrontal_part3'
  7: b'caudalmiddlefrontal_part4'
  8: b'corpuscallosum_part1'
  9: b'cuneus_part1'

First 10 right hemisphere region names:
  0: b'unknown_part1'
  1: b'bankssts_part1'
  2: b'bankssts_part2'
  3: b'caudalanteriorcingulate_part1'
  4: b'caudalmiddlefrontal_part1'
  5: b'caudalmiddlefrontal_part2'
  6: b'caudalmiddlefrontal_part3'
  7: b'caudalmiddlefrontal_part4'
  8: b'corpuscallosum_part1'
  9: b'cuneus_part1'


**Understanding the annotation file structure:**
- `labels_lh/rh`: Array where each element is the region ID for that vertex
- `names_lh/rh`: List of region names corresponding to each ID
- The label value at each vertex position tells us which region that vertex belongs to

Let's decode the region names and create a comprehensive list:


In [27]:
# Decode byte strings and add hemisphere prefixes for clarity
names_lh_decoded = ['lh_' + name.decode('utf-8') for name in names_lh]
names_rh_decoded = ['rh_' + name.decode('utf-8') for name in names_rh]

# Combine all region names
all_roi_names = names_lh_decoded + names_rh_decoded

print(f"Total number of regions: {len(all_roi_names)}")
print(f"Left hemisphere regions: {len(names_lh_decoded)}")
print(f"Right hemisphere regions: {len(names_rh_decoded)}")

print("\nExample region names:")
for i, name in enumerate(all_roi_names[::50]):  # Every 50th region
    print(f"  {i*50}: {name}")


Total number of regions: 312
Left hemisphere regions: 154
Right hemisphere regions: 158

Example region names:
  0: lh_unknown_part1
  50: lh_lingual_part5
  100: lh_rostralmiddlefrontal_part1
  150: lh_insula_part1
  200: rh_lateralorbitofrontal_part3
  250: rh_precentral_part9
  300: rh_supramarginal_part3


## Understanding region centroids

A centroid is the geometric center of a region. Let's calculate and visualize centroids for some regions.


In [28]:
def calculate_region_centroid(vertices, labels, region_id):
    """Calculate the centroid (center point) of a specific region."""
    # Find all vertices that belong to this region
    region_vertices = vertices[labels == region_id]
    
    if len(region_vertices) == 0:
        return None
    
    # Calculate the mean of all vertex coordinates
    centroid = np.mean(region_vertices, axis=0)
    return centroid


In [29]:
# Calculate centroids for a few example regions
print("Region centroids (first 5 regions):")
for region_id in range(1):
    centroid_lh = calculate_region_centroid(pial_left.Points, labels_lh, region_id)
    centroid_rh = calculate_region_centroid(pial_right.Points, labels_rh, region_id)
    
    region_name_lh = names_lh_decoded[region_id] if region_id < len(names_lh_decoded) else "N/A"
    region_name_rh = names_rh_decoded[region_id] if region_id < len(names_rh_decoded) else "N/A"
    
    print(f"\nRegion {region_id}:")
    if centroid_lh is not None:
        print(f"  {region_name_lh}: {centroid_lh}")
    if centroid_rh is not None:
        print(f"  {region_name_rh}: {centroid_rh}")


Region centroids (first 5 regions):

Region 0:
  lh_unknown_part1: [-12.92104666 -14.84461824  -2.90817507]
  rh_unknown_part1: [ 14.45473246 -14.48470301  -2.68204723]


## Working with test data

Let's load some test data and understand how to map it to brain regions.


In [30]:
# Load test data
df_test = pd.read_csv('data/test.csv', header=None, names=['region_name', 'value'])
test_values = df_test['value'].values

print(f"Test data shape: {test_values.shape}")
print(f"Value range: {test_values.min():.3f} to {test_values.max():.3f}")
print(f"Mean value: {test_values.mean():.3f}")
print(f"Standard deviation: {test_values.std():.3f}")

Test data shape: (308,)
Value range: -1.690 to 1.680
Mean value: 0.008
Standard deviation: 0.975


In [31]:
# Show first few values
print("\nFirst 10 test values:")
for i, val in enumerate(test_values[:10]):
    print(f"  {i}: {val:.3f}")



First 10 test values:
  0: 0.457
  1: 0.072
  2: 0.240
  3: 1.420
  4: 0.330
  5: 1.110
  6: 1.250
  7: -1.510
  8: -1.560
  9: 0.251


## Filtering and preparing region data

Some regions in parcellation schemes are not useful for analysis (like corpus callosum, unknown regions). Let's filter them out.


In [32]:
# Define regions to exclude (these are typically not useful for cortical analysis)
excluded_rois = {
    'lh_unknown_part1', 'rh_unknown_part1', 
    'lh_corpuscallosum_part1', 'rh_corpuscallosum_part1'
}

# Find which regions are being excluded
excluded_found = [name for name in all_roi_names if name in excluded_rois]
print(f"Regions to be excluded: {excluded_found}")

# Create list of valid (real) regions
valid_roi_names = [name for name in all_roi_names if name not in excluded_rois]

print(f"\nTotal regions: {len(all_roi_names)}")
print(f"Valid regions (after exclusion): {len(valid_roi_names)}")
print(f"Excluded regions: {len(all_roi_names) - len(valid_roi_names)}")


Regions to be excluded: ['lh_unknown_part1', 'lh_corpuscallosum_part1', 'rh_unknown_part1', 'rh_corpuscallosum_part1']

Total regions: 312
Valid regions (after exclusion): 308
Excluded regions: 4


## Mapping data to brain vertices

This is a crucial step: we need to map our region-level data to individual vertices on the brain surface.


In [33]:
# Create a mapping from region names to values
# We'll map our test data to the valid regions
value_dict = {}
for i, region in enumerate(valid_roi_names):
    if i < len(test_values):
        value_dict[region] = test_values[i]
    else:
        value_dict[region] = np.nan  # Fill missing regions with NaN

print(f"Value dictionary size: {len(value_dict)}")
print(f"Test values used: {len(test_values)}")
print(f"Regions with data: {sum(1 for v in value_dict.values() if not np.isnan(v))}")
print(f"Regions with NaN: {sum(1 for v in value_dict.values() if np.isnan(v))}")


Value dictionary size: 308
Test values used: 308
Regions with data: 308
Regions with NaN: 0


In [34]:
def map_values_to_vertices(labels, roi_names, value_dict, excluded_rois):
    """Map region-level values to individual vertices.
    
    Args:
        labels: Array of region IDs for each vertex
        roi_names: List of region names corresponding to IDs
        value_dict: Dictionary mapping region names to values
        excluded_rois: Set of region names to exclude
    
    Returns:
        Array of values for each vertex
    """
    vertex_values = []
    
    for label in labels:
        # Check if this is a valid region ID
        if 0 <= label < len(roi_names):
            region_name = roi_names[label]
            
            # Check if this region should be excluded
            if region_name not in excluded_rois:
                vertex_values.append(value_dict.get(region_name, np.nan))
            else:
                vertex_values.append(np.nan)
        else:
            vertex_values.append(np.nan)
    
    return np.array(vertex_values)

# Map values to vertices for both hemispheres
vertex_values_lh = map_values_to_vertices(labels_lh, all_roi_names, value_dict, excluded_rois)
vertex_values_rh = map_values_to_vertices(labels_rh, all_roi_names, value_dict, excluded_rois)

print(f"Left hemisphere vertex values:")
print(f"  Shape: {vertex_values_lh.shape}")
print(f"  Non-NaN values: {np.sum(~np.isnan(vertex_values_lh))}")
print(f"  Value range: {np.nanmin(vertex_values_lh):.3f} to {np.nanmax(vertex_values_lh):.3f}")

print(f"\nRight hemisphere vertex values:")
print(f"  Shape: {vertex_values_rh.shape}")
print(f"  Non-NaN values: {np.sum(~np.isnan(vertex_values_rh))}")
print(f"  Value range: {np.nanmin(vertex_values_rh):.3f} to {np.nanmax(vertex_values_rh):.3f}")


Left hemisphere vertex values:
  Shape: (163842,)
  Non-NaN values: 149953
  Value range: -1.690 to 1.680

Right hemisphere vertex values:
  Shape: (163842,)
  Non-NaN values: 148752
  Value range: -1.690 to 1.680


## Adding data to brain surfaces

Now we'll attach our vertex-level data to the brain surface objects so they can be visualized.


In [35]:
# Add our data to the brain surfaces
# The 'p' parameter means we're adding point data (vertex-level data)
pial_left.append_array(vertex_values_lh, name='test_data', at='p')
pial_right.append_array(vertex_values_rh, name='test_data', at='p')

print("Data successfully added to brain surfaces.")
print(f"Left surface data shape: {pial_left.PointData['test_data'].shape}")
print(f"Right surface data shape: {pial_right.PointData['test_data'].shape}")


Data successfully added to brain surfaces.
Left surface data shape: (163842,)
Right surface data shape: (163842,)


## Basic single-view sisualization

Let's start with simple single-view visualizations to understand the basics.


In [36]:
# Create a simple single-view plot (left hemisphere, lateral view)
plot_surf(
    {'lh': pial_left},  # Surface as dictionary with key
    [['lh']],  # Layout specification for single view
    array_name=[['test_data']],  # Data array to visualize
    cmap='viridis',  # Color map
    nan_color=(0.7, 0.7, 0.7, 1),  # Color for NaN values (gray)
    color_bar=True,  # Show color bar
    zoom=1.2,  # Zoom level
    interactive=False,  # Don't open interactive window
    screenshot=True,  # Save as image
    transparent_bg=False,
    filename='figures/single_view_lateral.png',
    size=(1000, 1000))


'/Users/apple/Tutorials/visualizing_neuroimaging/figures/single_view_lateral.png'

In [37]:
# Try a different view - medial (inside) view
plot_surf(
    {'lh': pial_left},  # Surface as dictionary with key
    [['lh']],  # Layout specification for single view
    array_name=[['test_data']],  # Data array to visualize
    view=[['medial']],  # Medial view shows the inside of the hemisphere
    cmap='viridis',
    nan_color=(0.7, 0.7, 0.7, 1),
    color_bar=True,
    zoom=1.2,
    interactive=False,
    screenshot=True,
    transparent_bg=False,
    filename='figures/single_view_medial.png',
    size=(1000, 1000))


'/Users/apple/Tutorials/visualizing_neuroimaging/figures/single_view_medial.png'

## Different color schemes

Let's explore different color maps to see how they affect the visualization.


In [38]:
# Try different color maps
color_maps = ['viridis', 'plasma', 'inferno', 'coolwarm', 'RdYlBu_r']

for cmap in color_maps:
    plot_surf(
        {'lh': pial_left},  # Surface as dictionary with key
        [['lh']],  # Layout specification for single view
        array_name=[['test_data']],  # Data array to visualize
        view=[['lateral']],  # View specification
        cmap=cmap,
        nan_color=(0.7, 0.7, 0.7, 1),
        color_bar=True,
        zoom=1.2,
        interactive=False,
        screenshot=True,
        transparent_bg=False,
        filename=f'figures/color_map_{cmap}.png',
        size=(1000, 1000)
    )
    print(f"Color map '{cmap}' saved to figures/color_map_{cmap}.png")

Color map 'viridis' saved to figures/color_map_viridis.png
Color map 'plasma' saved to figures/color_map_plasma.png
Color map 'inferno' saved to figures/color_map_inferno.png
Color map 'coolwarm' saved to figures/color_map_coolwarm.png
Color map 'RdYlBu_r' saved to figures/color_map_RdYlBu_r.png


## Two-view visualization

Now let's create side-by-side views to compare different perspectives.


In [39]:
# Create a two-view plot (lateral and medial of left hemisphere)
plot_surf(
    {'lh': pial_left},  # Surface as dictionary with key
    [['lh', 'lh']],  # Two views side by side (both using 'lh' key)
    array_name=[['test_data']],  # Data array for both views
    view=[['lateral', 'medial']],  # View specifications for each panel
    cmap='viridis',
    nan_color=(0.7, 0.7, 0.7, 1),
    color_bar=True,
    zoom=1.2,
    interactive=False,
    screenshot=True,
    transparent_bg=False,
    filename='figures/two_view_lh.png',
    size=(1400, 1000))


'/Users/apple/Tutorials/visualizing_neuroimaging/figures/two_view_lh.png'

In [40]:
# Create a two-view plot comparing left and right hemispheres
plot_surf(
    {'lh': pial_left, 'rh': pial_right},  # Both hemispheres
    [['lh', 'rh']],  # Left and right side by side
    array_name=[['test_data']],  # Same data for both
    view=[['lateral', 'lateral']],  # Both in lateral view
    cmap='viridis',
    nan_color=(0.7, 0.7, 0.7, 1),
    color_bar=True,
    zoom=1.2,
    interactive=False,
    screenshot=True,
    transparent_bg=False,
    filename='figures/two_view_hemispheres.png',
    size=(1400, 1000)
)

print("Two-view plot (both hemispheres) saved to figures/two_view_hemispheres.png")


Two-view plot (both hemispheres) saved to figures/two_view_hemispheres.png


## Four-view visualization

The most comprehensive view shows all four perspectives: left lateral, left medial, right lateral, and right medial.


In [41]:
# Create the comprehensive four-view plot
plot_surf(
    {'lh': pial_left, 'rh': pial_right},  # Both hemispheres
    [['lh', 'lh', 'rh', 'rh']],  # Left, left, right, right
    array_name=[['test_data']],  # Data array
    view=[['lateral', 'medial', 'lateral', 'medial']],  # Views for each panel
    cmap='viridis',
    nan_color=(0.7, 0.7, 0.7, 1),
    color_bar=True,
    zoom=1.2,
    interactive=False,
    screenshot=True,
    transparent_bg=False,
    filename='figures/four_view_comprehensive.png',
    size=(1400, 1000))


'/Users/apple/Tutorials/visualizing_neuroimaging/figures/four_view_comprehensive.png'

## Custom color bar

Sometimes you want more control over the color bar. Let's create a custom one.


In [42]:
# Create a plot without the built-in color bar
plot_surf(
    {'lh': pial_left, 'rh': pial_right},
    [['lh', 'lh', 'rh', 'rh']],
    array_name=[['test_data']],
    view=[['lateral', 'medial', 'lateral', 'medial']],
    cmap='viridis',
    nan_color=(0.7, 0.7, 0.7, 1),
    color_bar=False,  # Disable built-in color bar
    zoom=1.2,
    interactive=False,
    screenshot=True,
    transparent_bg=False,
    filename='figures/brain_without_colorbar.png',
    size=(1400, 1000))

# Create a custom color bar
valid_values = test_values[~np.isnan(test_values)]
vmin = np.min(valid_values)
vmax = np.max(valid_values)

fig, cbar_ax = plt.subplots(figsize=(0.8, 3.0), dpi=300)
norm = plt.Normalize(vmin=vmin, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, ax=cbar_ax, orientation='vertical', shrink=1.0, aspect=10)
cbar.set_ticks([vmin, vmax])
cbar.set_ticklabels([f'{vmin:.2f}', f'{vmax:.2f}'])
cbar.ax.tick_params(labelsize=9)
cbar_ax.remove()

# Save the custom color bar
plt.savefig('figures/custom_colorbar.png', bbox_inches='tight', dpi=300, facecolor='white')
plt.close()

In [ ]:
# Combine the brain image with the custom color bar
brain_img = Image.open('figures/brain_without_colorbar.png')
cbar_img = Image.open('figures/custom_colorbar.png')

# Create final composite image
final_width = brain_img.size[0] + cbar_img.size[0] + 20
final_height = brain_img.size[1]
final_image = Image.new('RGB', (final_width, final_height), (255, 255, 255))

# Paste brain image
final_image.paste(brain_img, (0, 0))

# Paste color bar on the right
cbar_x = brain_img.size[0] + 20
cbar_y = (final_height - cbar_img.size[1]) // 2
final_image.paste(cbar_img, (cbar_x, cbar_y))

# Save the final composite
final_image.save('figures/final_composite.png', dpi=(300, 300))


### Next:
- Try different parcellation schemes
- Experiment with different data types
- Explore interactive visualization tools

